# Phase 1 project

This note book is designed to analyse data from the six different movie companies and decide which is best type of content the users consume for a company that is new to the movie making scene.

The analysis will be done in-order to answer the following questions:

* Which are the best performing movies categories to produce?
* Which demographics watch movies ?
* What each demographic prefer to watch ?

Specifically, this will cover:

* Using pandas to filter data
* Using scipy.stats to calcluate probalility and find suitability of data for recommendation

The analysis will divide each data source to a module within a note book. Data cleaning will done comprehensively then afterwords model will be selected to analysis the data so us to answer the business questions above.The modules are:

 #### 1 BOM movie company analysis
 #### 2 


## Movie Budget Analysis

In [2]:
#Import the necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [36]:
#Import data from the datasource using pandas and showing the first 5 columns
df_movie_budgets=pd.read_csv('tn_movie_budgets.csv')
df_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [48]:
# Getting more info about the data to gain a better understanding 
df_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  0 non-null      float64
 4   domestic_gross     0 non-null      float64
 5   worldwide_gross    0 non-null      float64
dtypes: float64(3), int64(1), object(2)
memory usage: 271.2+ KB


#### Data EDA and cleanup
From the info gotten above, we can see that the dataset has 5782 rows and 6 columns.

In the cell below we will check for null/missing values and duplicate values from the datasource.

In the production_budget, domestic_gross and worldwide_gross columns, I will remove currency sign and commas to make them fully numeric

In [33]:
# Checking for null values
na_values = df_movie_budgets.isnull().sum()
print(na_values)

# The data source has no null values on any of the columns

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64


In [39]:
# Checking for duplicate values
duplicates = df_movie_budgets[df_movie_budgets.duplicated()].sum()
print(duplicates)

#from the analysis seen below the data also has no duplicate values

id                   0.0
release_date         0.0
movie                0.0
production_budget    0.0
domestic_gross       0.0
worldwide_gross      0.0
dtype: float64


### Defining null and alternate hypothesis

 #### Null Hypotheis(H0 )
 
 The 